230513 23:30

10 epoch - submission(14).csv -> 0.02683

100 epoch - version 3 -> 0.0163

cropping + CNN
```
autoencoder = keras.Sequential([
    layers.Input(shape=img_dim),
    # encoder
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    
    layers.Dropout(0.5),

    # decoder
    layers.Conv2DTranspose(128, (3, 3), activation='relu'),
    
    layers.Conv2DTranspose(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    
    layers.Conv2DTranspose(64, (3, 3), activation='relu'),
    layers.Conv2D(1, (1, 1), activation='sigmoid')
])
```

In [ ]:
import zipfile
import os
from collections import Counter

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math

import keras
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Import Data

### Unzip images into workspace directories

In [ ]:
path_zip = '/kaggle/input/denoising-dirty-documents/'
path = '/kaggle/working/'

with zipfile.ZipFile(path_zip + 'train.zip', 'r') as zip_ref:
    zip_ref.extractall(path)

with zipfile.ZipFile(path_zip + 'test.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  
    
with zipfile.ZipFile(path_zip + 'train_cleaned.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  
    
# list of image names
train_img = sorted(os.listdir(path + '/train'))
train_cleaned_img = sorted(os.listdir(path + '/train_cleaned'))
test_img = sorted(os.listdir(path + '/test'))

crop to 140 (height, 420/3) x 135(width,540/4)

## Preprocess Image

In [ ]:
img_dim = (140, 135, 1)

In [ ]:
def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_np = np.asarray(img, dtype="float32") 
    img_np /= 255.0
    img_np = np.reshape(img_np, img_dim) # expand to 3d
    return img_np

In [ ]:
def crop_image(img):
    img_h, img_w = img.shape[0], img.shape[1]
    
    # center image by adding white paddings
    UD_border = img_dim[0] * ((img_h - 1) // img_dim[0] + 1) - img_h
    U, D = UD_border // 2, math.ceil(UD_border / 2)
    
    LR_border = img_dim[1] * ((img_w - 1) // img_dim[1] + 1) - img_w
    L, R = LR_border // 2, math.ceil(LR_border / 2)
    
    padded_img = cv2.copyMakeBorder(img, U, D, L, R, cv2.BORDER_CONSTANT, value=(255, 255, 255))
    
    # crop image
    img_list = []
    no_row, no_col = (img_h + UD_border) // img_dim[0], (img_w + LR_border) // img_dim[1]
    for row in range(no_row):
        for col in range(no_col):
            r1, r2 = row * img_dim[0], (row + 1) * img_dim[0]
            c1, c2 = col * img_dim[1], (col + 1) * img_dim[1]
            crop_img = padded_img[r1:r2, c1:c2]
            img_list.append(crop_img)
    
    return img_list

In [ ]:
def load_img(path):
    # load image
    raw_img = cv2.imread(path)
    
    # crop image
    img_list = crop_image(raw_img)
    
    # preprocess - to gray, normalize, to 3d
    processed_list = []
    for img in img_list:
        processed_img = preprocess_image(img)
        processed_list.append(processed_img)
        
    return processed_list

In [ ]:
def print_imgs(img_list):
    n = len(img_list)
    for i in range(n):
        plt.subplot(math.ceil(n/4),4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img_list[i], cmap='gray')

In [ ]:
# preprocess images
train = []
train_cleaned = []
test = []

for f in sorted(os.listdir(path + 'train/')):
    processed_img = load_img(path + 'train/' + f)
    for img in processed_img:
        train.append(img)

for f in sorted(os.listdir(path + 'train_cleaned/')):
    processed_img = load_img(path + 'train_cleaned/' + f)
    for img in processed_img:
        train_cleaned.append(img)

In [ ]:
print_imgs(train[4:8])

In [ ]:
print_imgs(train_cleaned[4:8])

## Train test split
The data size is small, so use a smaller portion of validation set

In [ ]:
X_train = np.asarray(train)
y_train = np.asarray(train_cleaned)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
print(X_train.shape)

# Prepare Autoencoder

In [ ]:
autoencoder = keras.Sequential([
    layers.Input(shape=img_dim),
    # encoder
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    
    layers.Dropout(0.5),

    # decoder
    layers.Conv2DTranspose(128, (3, 3), activation='relu'),
    
    layers.Conv2DTranspose(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    
    layers.Conv2DTranspose(64, (3, 3), activation='relu'),
    layers.Conv2D(1, (1, 1), activation='sigmoid')
])

In [ ]:
print(autoencoder.summary())

In [ ]:
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['mae']
                   )

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=15)

hist = autoencoder.fit(X_train, y_train,
                epochs=300,
                batch_size=16,
                shuffle=True,
                validation_data=(X_val, y_val),
                callbacks=[callback],
               )

In [ ]:
# Check how loss & mae went down
epoch_loss = hist.history['loss']
epoch_val_loss = hist.history['val_loss']
epoch_mae = hist.history['mae']
epoch_val_mae = hist.history['val_mae']

plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(range(0,len(epoch_loss)), epoch_loss, 'b-', linewidth=2, label='Train Loss')
plt.plot(range(0,len(epoch_val_loss)), epoch_val_loss, 'r-', linewidth=2, label='Val Loss')
plt.title('Evolution of loss on train & validation datasets over epochs')
plt.legend(loc='best')

plt.subplot(1,2,2)
plt.plot(range(0,len(epoch_mae)), epoch_mae, 'b-', linewidth=2, label='Train MAE')
plt.plot(range(0,len(epoch_val_mae)), epoch_val_mae, 'r-', linewidth=2,label='Val MAE')
plt.title('Evolution of MAE on train & validation datasets over epochs')
plt.legend(loc='best')

plt.show()

# Inference
note: we need to crop the image -> prediction -> reconstruction

In [ ]:
def reconstruct_img(img_list, path):
    # obtain original image and borders dimensions
    img = cv2.imread(path)
    img_h, img_w = img.shape[0], img.shape[1]
    
    UD_border = img_dim[0] * ((img_h - 1) // img_dim[0] + 1) - img_h
    U, D = UD_border // 2, math.ceil(UD_border / 2)
    
    LR_border = img_dim[1] * ((img_w - 1) // img_dim[1] + 1) - img_w
    L, R = LR_border // 2, math.ceil(LR_border / 2)
    
    no_row, no_col = (img_h + UD_border) // img_dim[0], (img_w + LR_border) // img_dim[1]
    
#     print("image size", img.shape)
#     print("paddings", U, D, L, R)
    
    # concatenate the grids back into one image
    # concat by row first
    row_imgs = []
    for row in range(no_row):
        im_r = cv2.hconcat([img_list[row * no_col + col] for col in range(no_col)])
#         print(im_r.shape)
        row_imgs.append(im_r)
        
    # concat all rows to form 1 image
    padded_img = cv2.vconcat(row_imgs)
            
    # remove paddings
    if D > 0 and R > 0:
        crop_img = padded_img[U:-D, L:-R]
    elif D > 0:
        crop_img = padded_img[U:-D]
    elif R > 0:
        crop_img = padded_img[:, L:-R]
    else:
        crop_img = padded_img
        
#     print(padded_img.shape, crop_img.shape)
    return crop_img

In [ ]:
y_test = []
c = 0

for f in sorted(os.listdir(path + 'test/')):
    img_path = path + 'test/' + f
    
    # predict
    processed_img = np.asarray(load_img(img_path))
    pred_arr = autoencoder.predict(processed_img, batch_size=16, verbose=0)
    
    # process the output
    pred_list = [pred_arr[i] for i in range(pred_arr.shape[0])]
    processed_output = reconstruct_img(pred_list, img_path)
    y_test.append(processed_output)
    
#     print(processed_output.shape)
#     c += 1
#     if c > 4:
#         break
print(len(y_test))
#     plt.imshow(processed_output, cmap='gray')


In [ ]:
# it will take a while!
ids = []
vals = []
for i, f in enumerate(test_img):
    file = path + 'test/' + f
    imgid = int(f[:-4])
    img = cv2.imread(file, 0)
    img_shape = img.shape
    # print('Processing image: {} \tinto size: {}'.format(f, img_shape))    # uncomment to see progress
    preds_reshaped = cv2.resize(y_test[i], (img_shape[1], img_shape[0]))

    for r in range(img_shape[0]):
        for c in range(img_shape[1]):
            ids.append(str(imgid)+'_'+str(r + 1)+'_'+str(c + 1))
            vals.append(preds_reshaped[r, c])

submission = pd.DataFrame({'id': ids, 'value': vals})
submission.to_csv('submission.csv',index = False)

print('Results saved to submission.csv!')
print('Length of IDs: {}'.format(len(ids)))

In [ ]:
import shutil
shutil.rmtree(path + 'train/')
shutil.rmtree(path + 'test/')
shutil.rmtree(path + 'train_cleaned/')

In [ ]:
df = pd.read_csv("/kaggle/working/submission.csv")